<br><br>
<a id='Header' style="font-size:35px; color:#1D2951; text-decoration:none; text-align:center;"><center><b>Analytics Engineer
Technical Task</b></center></a>
<br/><br/>



<a id='Header' style="font-size:30px; color:#1D2951; text-decoration:none">
    <b> Table of Contents </b>
</a> 
<ul>
<li><a href="#intro" style="font-size:15px; color:#1D2951">Introduction</a></li>
<li><a href="#eda" style="font-size:15px; color:#1D2951">Operations Analysis</a></li>
<li><a href="#Q1" style="font-size:15px; color:#1D2951">How many trips were performed by our delivery agents</a></li>
<li><a href="#Q2" style="font-size:15px; color:#1D2951">The average number of orders delivered per trip</a></li>
<li><a href="#Q3" style="font-size:15px; color:#1D2951">The average number of trips per day per delivery agent</a></li>
</ul>
<br>
<hr>

<a id='intro' style="font-size:25px; color:#1D2951; text-decoration:none"><b>Introduction</b></a>

Here we will be examining delivery agent delivery data to understand how well the delivery agents are performing. We will focus on a few key metrics: 
1. Total trips: (Determining the total number of trips made by delivery agents).
2. Orders per trip: (Calculating the average number of orders delivered per trip).
3. Agent trips per day: (Figuring out how many trips on average for each delivery agent makes in a day).

where the `full trip` is defined as
 > when the delivery agent leaves the dark store & ends when the delivery agent returns back to the dark store


For the rest of the notebook, we will be heavily using `pandasql (SQLite)`.

In [1]:
from pandasql import sqldf
import pandasql as ps
import pandas as pd
import numpy as np
import datetime as dt


import warnings
warnings.filterwarnings('ignore')

<a id='dataset' style="font-size:25px; color:#1D2951; text-decoration:none"><b>Dataset Overview</b></a>

In [2]:
data = pd.read_csv('analytics_task_data.csv')

In [3]:
data.head()

,order_id,order_status,delivery_person_id,created_at_eet,received_at_eet,started_at_eet,arrived_at_eet,delivered_at_eet
0,6785158,completed,378483,10/1/2022 20:02,10/1/2022 20:45,10/1/2022 20:50,10/1/2022 20:54,10/1/2022 20:56
1,6772097,completed,287306,10/1/2022 1:37,10/1/2022 1:44,10/1/2022 1:44,10/1/2022 1:45,10/1/2022 1:46
2,6772445,completed,438128,10/1/2022 4:09,10/1/2022 4:16,10/1/2022 4:25,10/1/2022 4:30,10/1/2022 4:31
3,6771722,completed,444472,10/1/2022 0:19,10/1/2022 0:24,10/1/2022 0:24,10/1/2022 0:28,10/1/2022 0:34
4,6772199,completed,438128,10/1/2022 2:09,10/1/2022 2:18,10/1/2022 2:19,10/1/2022 2:21,10/1/2022 2:27


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015 entries, 0 to 1014
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   order_id            1015 non-null   int64 
 1   order_status        1015 non-null   object
 2   delivery_person_id  1015 non-null   int64 
 3   created_at_eet      1015 non-null   object
 4   received_at_eet     1009 non-null   object
 5   started_at_eet      1008 non-null   object
 6   arrived_at_eet      1007 non-null   object
 7   delivered_at_eet    993 non-null    object
dtypes: int64(2), object(6)
memory usage: 63.6+ KB


In [5]:
date_cols = ['created_at_eet', 'received_at_eet', 'started_at_eet', 'arrived_at_eet', 'delivered_at_eet']
id_cols = ['order_id', 'delivery_person_id']

data[date_cols] = data[date_cols].apply(pd.to_datetime)
data[id_cols] = data[id_cols].astype(str)

In [6]:
# data[data['order_status'] == 'not-received']

In [7]:
# check order status distribution
data['order_status'].value_counts()

completed       993
not-received     16
cancelled         6
Name: order_status, dtype: int64

In [8]:
# num of unique delivery agent
data['delivery_person_id'].nunique()

58

In [9]:
# count of null values in 'started_at_eet'
data['started_at_eet'].isnull().sum()

7

---

**!IMPORTANT,**

    - A trip starts when the delivery agent receives orders to deliver and make the first delivery (aka first `started_at` for each trip)
    
    - There is no datetime for when the delivery agent returns to the dark store, so there is no guarantee that the agent did already return to the store except for when the agent has another delivery. 
    -> THUS, for all the upcoming analysis we will assume that each trip is conducted and counted in the moment the agent starts the first customer delivery (aka first `started_at` for each trip). this is valid for almost all cases except the last delivery which will be counted as a full trip even though the agent has not returned yet. (limitation)
    
    - The trip doesn't count in the `order_status`, so the trip is called a trip even if the customer has not received the item (whether the customer has cancelled it or any other scenario.)
    
    - Orders that has not been delivered (`started_at == null`) are filtered out, this is for the sake of this analysis, thus we can count out the trips based on the first receive at the store as we guarantee that the order will eventually start.

In [10]:
query = '''
        WITH base AS
           (SELECT  order_id,
                    delivery_person_id,
                    'RECEIVED' AS action,
                    received_at_eet AS time,
                    order_status
            FROM data
            WHERE started_at_eet IS NOT NULL

            UNION ALL
            SELECT  order_id,
                    delivery_person_id,
                    'STARTED' AS action,
                    started_at_eet AS time,
                    order_status
            FROM data
            WHERE started_at_eet IS NOT NULL

            ),
            
        lagged_data AS
       (SELECT  order_id,
                delivery_person_id,
                action,
                time,
                order_status,
                CASE WHEN action = 'RECEIVED' THEN 1 ELSE 0 END AS action_as_label,
            --   LAG(action_as_label) OVER(PARTITION BY delivery_person_id ORDER BY time) AS action_as_label_lag
                LAG(CASE WHEN action = 'RECEIVED' THEN 1 ELSE 0 END) OVER(PARTITION BY delivery_person_id ORDER BY time) AS action_as_label_lag
                
        FROM base
        ORDER BY time ASC),
        
        main AS
        (SELECT *,               
                CASE WHEN action_as_label_lag IS NULL THEN action_as_label  --
                     WHEN action_as_label = 1 AND action_as_label_lag = 1 THEN 0
                     WHEN action_as_label = 1 AND action_as_label_lag = 0 THEN 1
                     WHEN action_as_label = 0 THEN 0
                     ELSE action_as_label  END AS start_trip_flag
                     
        FROM lagged_data),
        
        trip_id_data AS
        (SELECT *, 
                SUM(start_trip_flag) OVER(PARTITION BY delivery_person_id ORDER BY time) AS trip_id
        FROM main)
            
    SELECT  *
    FROM trip_id_data
    where delivery_person_id = 291322
    LIMIT 20
'''
df = ps.sqldf(query)
df.head(15)

,order_id,delivery_person_id,action,time,order_status,action_as_label,action_as_label_lag,start_trip_flag,trip_id
0,6771704,291322,RECEIVED,2022-10-01 00:28:00.000000,completed,1,NaN,1,1
1,6771790,291322,RECEIVED,2022-10-01 00:31:00.000000,completed,1,1.0,0,1
2,6771790,291322,STARTED,2022-10-01 00:32:00.000000,completed,0,1.0,0,1
3,6771704,291322,STARTED,2022-10-01 00:47:00.000000,completed,0,0.0,0,1
4,6772027,291322,RECEIVED,2022-10-01 01:31:00.000000,completed,1,0.0,1,2
5,6772027,291322,STARTED,2022-10-01 01:32:00.000000,completed,0,1.0,0,2
6,6772161,291322,RECEIVED,2022-10-01 02:02:00.000000,completed,1,0.0,1,3
7,6772161,291322,STARTED,2022-10-01 02:02:00.000000,completed,0,1.0,0,3
8,6772435,291322,RECEIVED,2022-10-01 04:10:00.000000,completed,1,0.0,1,4
9,6772443,291322,RECEIVED,2022-10-01 04:12:00.000000,completed,1,1.0,0,4



we introduced various variables that would make the analysis much easier,
    
    - First, to figure out when the agent starts the trip, we concatenated both `received_at` and `started_at` columns into one column `time` and sorted them out and labeled them by a new column `action` which differentiate between which is the `RECEIVED` date and which is the date for starting the delivery `STARTED`

    - and made `start_trip_flag` that indicates when the agent starts a trip, that takes in count if agent received multiple orders at the store and made a trip for multiple customer deliveries in the same trip.

    - the logic simply (given that all orders that has not been delivered are filtered out), 
    when the agent receives the first order at the store we count it as a trip, and count the second trip when another order is received after making any `STARTED` delivery and label all orders in between in the same trip.

---

<a id='eda' style="font-size:24px; color:#1D2951; text-decoration:none"><b>Operations Analysis</b></a>

<a id='Q1' style="font-size:18px; color:#1D2951; text-decoration:none"><b>1. How many trips were performed by our delivery agents</b></a>

In [11]:
query = '''
        WITH base AS
           (SELECT  order_id,
                    delivery_person_id,
                    'RECEIVED' AS action,
                    received_at_eet AS time,
                    order_status
            FROM data
            WHERE started_at_eet IS NOT NULL

            UNION ALL
            SELECT  order_id,
                    delivery_person_id,
                    'STARTED' AS action,
                    started_at_eet AS time,
                    order_status
            FROM data
            WHERE started_at_eet IS NOT NULL

            ),
            
        lagged_data AS
       (SELECT  order_id,
                delivery_person_id,
                action,
                time,
                order_status,
                CASE WHEN action = 'RECEIVED' THEN 1 ELSE 0 END AS action_as_label,
            --   LAG(action_as_label) OVER(PARTITION BY delivery_person_id ORDER BY time) AS action_as_label_lag
                LAG(CASE WHEN action = 'RECEIVED' THEN 1 ELSE 0 END) OVER(PARTITION BY delivery_person_id ORDER BY time) AS action_as_label_lag
                
        FROM base
        ORDER BY time ASC),
        
        main AS
        (SELECT *,               
                CASE WHEN action_as_label_lag IS NULL THEN action_as_label
                     WHEN action_as_label = 1 AND action_as_label_lag = 1 THEN 0
                     WHEN action_as_label = 1 AND action_as_label_lag = 0 THEN 1
                     WHEN action_as_label = 0 THEN 0
                     ELSE action_as_label  END AS start_trip_flag
                     
        FROM lagged_data),
        
        trip_id_data AS
        (SELECT *,
                (CAST(delivery_person_id AS text) ||'-'|| CAST(SUM(start_trip_flag) OVER(PARTITION BY delivery_person_id ORDER BY time) AS text)) AS trip_id
        FROM main)
        
        SELECT COUNT(DISTINCT trip_id) AS num_trips
        FROM trip_id_data
        
'''
x = ps.sqldf(query)
x


,num_trips
0,710


<a id='Q2' style="font-size:18px; color:#1D2951; text-decoration:none"><b>2. The average number of orders delivered per trip</b></a>

In [12]:
query = '''
        WITH base AS
           (SELECT  order_id,
                    delivery_person_id,
                    'RECEIVED' AS action,
                    received_at_eet AS time,
                    order_status
            FROM data
            WHERE started_at_eet IS NOT NULL

            UNION ALL
            SELECT  order_id,
                    delivery_person_id,
                    'STARTED' AS action,
                    started_at_eet AS time,
                    order_status
            FROM data
            WHERE started_at_eet IS NOT NULL),
            
        lagged_data AS
       (SELECT  order_id,
                delivery_person_id,
                action,
                time,
                order_status,
                CASE WHEN action = 'RECEIVED' THEN 1 ELSE 0 END AS action_as_label,
            --   LAG(action_as_label) OVER(PARTITION BY delivery_person_id ORDER BY time) AS action_as_label_lag
                LAG(CASE WHEN action = 'RECEIVED' THEN 1 ELSE 0 END) OVER(PARTITION BY delivery_person_id ORDER BY time) AS action_as_label_lag
                
        FROM base
        ORDER BY time ASC),
        
        main AS
        (SELECT *,               
                CASE WHEN action_as_label_lag IS NULL THEN action_as_label
                     WHEN action_as_label = 1 AND action_as_label_lag = 1 THEN 0
                     WHEN action_as_label = 1 AND action_as_label_lag = 0 THEN 1
                     WHEN action_as_label = 0 THEN 0
                     ELSE action_as_label  END AS start_trip_flag
                     
        FROM lagged_data),
        
        trip_id_data AS
        (SELECT *, 
                (CAST(delivery_person_id AS text) ||'-'|| CAST(SUM(start_trip_flag) OVER(PARTITION BY delivery_person_id ORDER BY time) AS text)) AS trip_id
        FROM main),
        
        total_orders AS
            (SELECT trip_id,
                    COUNT(DISTINCT order_id) AS num_orders
            
             FROM trip_id_data
             WHERE order_status = 'completed'
             GROUP BY 1)
             
        SELECT AVG(num_orders) AS avg_orders_per_trip
        FROM total_orders        
    '''

x = ps.sqldf(query)
x

,avg_orders_per_trip
0,1.469417


<a id='Q3' style="font-size:18px; color:#1D2951; text-decoration:none"><b>3. The average number of trips per day per delivery agent</b></a>

Here I thought it would be insightful to also include the daily average across all agents as well as the average of agents individually. this will point out the contribution of each agent per day across all agent's colleagues

**This query calculates the average for each delivery agent separately, considering only their overall number of trips regarless of other agents.**

In [13]:
query = '''
        WITH base AS
           (SELECT  order_id,
                    delivery_person_id,
                    'RECEIVED' AS action,
                    received_at_eet AS time,
                    order_status
            FROM data
            WHERE started_at_eet IS NOT NULL

            UNION ALL
            SELECT  order_id,
                    delivery_person_id,
                    'STARTED' AS action,
                    started_at_eet AS time,
                    order_status
            FROM data
            WHERE started_at_eet IS NOT NULL

            ),
            
        lagged_data AS
       (SELECT  order_id,
                delivery_person_id,
                action,
                time,
                order_status,
                CASE WHEN action = 'RECEIVED' THEN 1 ELSE 0 END AS action_as_label,
            --   LAG(action_as_label) OVER(PARTITION BY delivery_person_id ORDER BY time) AS action_as_label_lag
                LAG(CASE WHEN action = 'RECEIVED' THEN 1 ELSE 0 END) OVER(PARTITION BY delivery_person_id ORDER BY time) AS action_as_label_lag
                
        FROM base
        ORDER BY time ASC),
        
        main AS
        (SELECT *,               
                CASE WHEN action_as_label_lag IS NULL THEN action_as_label
                     WHEN action_as_label = 1 AND action_as_label_lag = 1 THEN 0
                     WHEN action_as_label = 1 AND action_as_label_lag = 0 THEN 1
                     WHEN action_as_label = 0 THEN 0
                     ELSE action_as_label  END AS start_trip_flag
                     
        FROM lagged_data),

        trip_id_data AS
            (SELECT *, 
                    (CAST(delivery_person_id AS text) ||'-'|| CAST(SUM(start_trip_flag) OVER(PARTITION BY delivery_person_id ORDER BY time) AS text)) AS trip_id
            FROM main),

        trips_per_day_agent AS
            (SELECT  DATE(time) AS day,
                     delivery_person_id,
                     COUNT(DISTINCT trip_id) AS num_trips

             FROM trip_id_data
             WHERE order_status =('completed') -- NOT IN('cancelled','not-received')
             GROUP BY 1,2),
             
        total_trips_day AS
            (SELECT  day,
                    delivery_person_id,
                    SUM(num_trips) AS total_trips

            FROM trips_per_day_agent
            --WHERE delivery_person_id = 279462 -- 12.0 -- 279462
            GROUP BY 1,2)
            
        SELECT  delivery_person_id,
                AVG(total_trips) as avg_total_trips
        FROM total_trips_day
        group by 1    
    '''

x = ps.sqldf(query)
x

,delivery_person_id,avg_total_trips
0,145126,14.0
1,152165,5.0
2,172327,10.0
3,223935,1.0
4,243024,5.0
5,279462,5.5
6,282743,16.0
7,282770,19.0
8,283470,10.0
9,284968,12.0


**This query computes the daily average across all agents, taking into account the total number of trips by all agents.**

In [14]:
query = '''
        WITH base AS
           (SELECT  order_id,
                    delivery_person_id,
                    'RECEIVED' AS action,
                    received_at_eet AS time,
                    order_status
            FROM data
            WHERE started_at_eet IS NOT NULL

            UNION ALL
            SELECT  order_id,
                    delivery_person_id,
                    'STARTED' AS action,
                    started_at_eet AS time,
                    order_status
            FROM data
            WHERE started_at_eet IS NOT NULL),
            
        lagged_data AS
       (SELECT  order_id,
                delivery_person_id,
                action,
                time,
                order_status,
                CASE WHEN action = 'RECEIVED' THEN 1 ELSE 0 END AS action_as_label,
            --   LAG(action_as_label) OVER(PARTITION BY delivery_person_id ORDER BY time) AS action_as_label_lag
                LAG(CASE WHEN action = 'RECEIVED' THEN 1 ELSE 0 END) OVER(PARTITION BY delivery_person_id ORDER BY time) AS action_as_label_lag
                
        FROM base
        ORDER BY time ASC),
        
        main AS
        (SELECT *,               
                CASE WHEN action_as_label_lag IS NULL THEN action_as_label
                     WHEN action_as_label = 1 AND action_as_label_lag = 1 THEN 0
                     WHEN action_as_label = 1 AND action_as_label_lag = 0 THEN 1
                     WHEN action_as_label = 0 THEN 0
                     ELSE action_as_label  END AS start_trip_flag
                     
        FROM lagged_data),

        trip_id_data AS
            (SELECT *, 
                    (CAST(delivery_person_id AS text) ||'-'|| CAST(SUM(start_trip_flag) OVER(PARTITION BY delivery_person_id ORDER BY time) AS text)) AS trip_id
            FROM main),

        trips_per_day_agent AS
            (SELECT  DATE(time) AS day,
                     delivery_person_id,
                     COUNT(DISTINCT trip_id) AS num_trips

             FROM trip_id_data
             WHERE order_status =('completed') -- NOT IN('cancelled','not-received')
             GROUP BY 1,2),
             
        total_trips_day AS
           (SELECT  day,
                    delivery_person_id,
                    SUM(num_trips) AS total_trips

            FROM trips_per_day_agent
            GROUP BY 1,2)
            
        SELECT  day,
                AVG(total_trips) AS avg_total_trips
        FROM total_trips_day
        GROUP BY 1
        
    '''
x = ps.sqldf(query)
x


,day,avg_total_trips
0,2022-10-01,12.125
1,2022-10-02,1.350


<a id='notes' style="font-size:22px; color:#1D2951; text-decoration:none"><b>Notes</b></a>

- This analysis is performed based on the available data and the given definitions and the abscence of the `trip_id`.
- The analysis could go much easier if the `trip_id` is constructed from the backend, the moment the agent starts his first trip after being in the warehouse.
